In [11]:
from pathlib import Path
from PDFAdapter import PDFAdapter
from FitDAG import FitDAG
from FitRunner import FitRunner
import time
import re
import copy

data_dir_path = Path().cwd().parents[0] / "data" / "sequential_fit"
files = [file for file in data_dir_path.glob("*.gr")]
order = [int(re.search(r"(\d+)K\.gr", file.name).group(1)) for file in files]
files = [file for _, file in sorted(zip(order, files))]

structure_path = Path().cwd().parents[0] / "data" / "Ni.cif"

# Configure the first file
inputs = {
    "profile_string": files[0].read_text(),
    "structure_string": structure_path.read_text(),
    "xmin": 1.5,
    "xmax": 50,
    "dx": 0.01,
    "qmax": 25.0,
    "qmin": 0.1,
}

payload = {
    "scale": 0.25,  # 0.26
    "a": 3.52,
    "Uiso_0": 0.0015,  # 0.0016
    "delta2": 0.65,  # 0.64
    "delta1" : 0.3,  # 0.3
    "delta2": 0.65,  # 0.64
    "qdamp": 0.030,  # 0.028
    "qbroad": 0.02,  # 0.023
}


dag = FitDAG()
dag.from_str("a->scale->qdamp->Uiso_0->delta2->all")

In [12]:
start_time = time.time()
runner = FitRunner()
dag, adapters_dict = runner.run_workflow(dag, PDFAdapter, [inputs], [payload])
end_time = time.time()
print(f"FitRunner took {end_time - start_time} seconds.")

FitRunner took 4.919573545455933 seconds.


In [3]:
inputs = {
    "profile_string": files[0].read_text(),
    "structure_string": structure_path.read_text(),
    "xmin": 1.5,
    "xmax": 50,
    "dx": 0.01,
    "qmax": 25.0,
    "qmin": 0.1,
}

another_dag = dag.clean_copy(with_payload=False, with_besides_str=True, with_same_id=False, instance_type="FitDAG")

parent_node_id = dag.get_node_by_name("all")
parent_node_id = list(parent_node_id.keys())[0]

child_node_id = another_dag.get_node_by_name("start")
child_node_id = list(child_node_id.keys())[0]
child_node_id = list(another_dag.successors(child_node_id))[0]

dag.merge_dag(another_dag, parent_node_id, child_node_id)


In [4]:
for node_id, node_content in dag.nodes(data=True):
    print(f"Node ID: {node_id}")
    print(f"Node Name: {node_content.get('name', 'N/A')}")
    print(f"Node root_node_id: {dag.get_input_source(node_id)}")
    print("-----")

Node ID: fb6409d9-d62e-4070-81e6-00a2275dcfc0
Node Name: a
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: c796b89d-7251-4264-81b6-d29028c303fe
Node Name: scale
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: dc3af589-741b-4098-804f-c1367b586b11
Node Name: qdamp
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: 4e60dcdd-cec3-4751-88de-451d9f015a9e
Node Name: Uiso_0
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: 6f5f4c2c-4a69-4663-9a75-0123f33f6a6b
Node Name: delta2
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: 60fe7413-ea33-4ff9-a734-87293582669d
Node Name: all
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: bc6192b2-eb2c-4a59-9e06-6473032cf657
Node Name: start
Node root_node_id: bc6192b2-eb2c-4a59-9e06-6473032cf657
-----
Node ID: e682b531-5323-4c99-a836-cf2fec773dcd
Node Name: a_1
Node root_node_id: 7fc6d7d1-a397-4de4-903f-cddc8524601b
-----
Node I

In [5]:
dag.render("merged_dag.html")

In [10]:
for name in dag.all_names:
    node_id = list(dag.get_node_by_name(name).keys())[0]
    input_source_id = dag.get_input_source(node_id)
    parent_id = dag.get_payload_source(node_id)
    print(
        f"name: {name} ".ljust(20) 
        + f"source: {dag.nodes[input_source_id]['name']}".ljust(30)
        + f"parent: {dag.nodes[parent_id]['name']}".ljust(30)
          ) 

name: a             source: start                 parent: start                 
name: scale         source: start                 parent: a                     
name: qdamp         source: start                 parent: scale                 
name: Uiso_0        source: start                 parent: qdamp                 
name: delta2        source: start                 parent: Uiso_0                
name: all           source: start                 parent: delta2                


ValueError: max() iterable argument is empty

In [7]:
inputs = [
    {
    "profile_string": files[0].read_text(),
    "structure_string": structure_path.read_text(),
    "xmin": 1.5,
    "xmax": 50,
    "dx": 0.01,
    "qmax": 25.0,
    "qmin": 0.1,
},
    {
    "profile_string": files[1].read_text(),
    "structure_string": structure_path.read_text(),
    "xmin": 1.5,
    "xmax": 50,
    "dx": 0.01,
    "qmax": 25.0,
    "qmin": 0.1,
},
]
start_time = time.time()
runner = FitRunner()
dag, adapters_dict = runner.run_workflow(dag, PDFAdapter, inputs, [payload,None])
end_time = time.time()
print(f"FitRunner took {end_time - start_time} seconds.")

FitRunner took 11.685925960540771 seconds.


In [13]:
b = []
a = [1,2,3,4]
[(a[i], i) for i in b]

[]

In [14]:
b = []
a = [1,2,3,4]
all([a[i] == b[i] for i in range(len(b))])

True